<a href="https://colab.research.google.com/github/luise15/aamemoria-deteccioncancergastrico/blob/main/clasificador_biclase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#data loading
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
#data preprocessing
    #input
    #demas layers
#model creation
    #convulucionales
    #flaterns
    #dense
    #model input-output
#model training
    #compiple(optimizer, loss, metrics)
    #fit(batch_size, epoch)

In [ ]:
#esto es de la introduccion a keras para ingenieros 
#https://keras.io/getting_started/intro_to_keras_for_engineers/

#clasificador binario con las capas de inceptionV3, capa final fullyconceted de 1 neurona de saluda sigmoidea.
import numpy as np
import tensorflow as tf 
from tensorflow import keras
#from tensorflow.keras import layers

'''
#data loading
path_db=''
dataset_tumoral_noTumoral=keras.preprocessing.image_dataset_from_directory(path_db,image_size=(299,299))
#data preprocessing
    #input layer 
'''
inputs = keras.Input(shape=(None, None,3))
    #rescaling image to [0,1]
x=keras.layers.Rescaling(scale=1.0/255)(inputs)
#model creation
    #aplicando capas convolucionales
x = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x=keras.layers.MaxPooling2D(pool_size=(3,3))(x)
x=keras.layers.Conv2D(filters=32,kernel_size=(3,3), activation='relu')(x)
    #aplicando un flat para que salga un vector plano
x=keras.layers.GlobalAveragePooling2D()(x)
    #aplicando ultima capa fullyconected
numero_de_clases=2
outputs=keras.layers.Dense(numero_de_clases, activation='softmax')(x)
#instanciando el modelo
clf_tejTumoral = keras.Model(inputs=inputs, outputs=outputs)
#ver resumen del modelo
clf_tejTumoral.summary()
#Model training 
    #model compilation
clf_tejTumoral.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3), loss=keras.losses.BinaryCrossentropy())
#entrenamiento
#clf_tejTumoral.fit(dataset_tumoral_noTumoral, epoch=30)
#historial = clf_tejTumoral.fit(dataset_tumoral_noTumoral, epoch=30)

#prediction

#y_pred=clf_tejTumoral(x_nuevo)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 rescaling (Rescaling)       (None, None, None, 3)     0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, None, None, 32)   0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 global_average_pooling2d (G  (None, 32)               0         
 lobalAveragePooling2D)                                      

In [ ]:
#escribir la red que quiero hacer:
#clasificador binario de tejido tumoral- tejido no tumoral.
#con la arquitectura inceptionv3 entrenada desde 0
#tamaño de parches 299x299
#clasificador binario con las capas de inceptionV3, capa final fullyconceted de 1 neurona de saluda sigmoidea.

In [ ]:
#data preprocessing
    #input
    #demas layers
#model creation
    #convulucionales
    #flaterns
    #dense
    #model instanciation
#model training
    #compiple(optimizer, loss, metrics)
    #fit(batch_size, epoch)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
start_time = time.time()

#data loading
#path_db='/home/luise/databases_memoria/db_biclase'
path_db='/content/drive/MyDrive/memoria/db_biclase'

class_names=['tejidonotumoral','tejidotumoral']


train_ds =tf.keras.preprocessing.image_dataset_from_directory(
    path_db,
    labels= "inferred",
    label_mode="binary",
    class_names=class_names,
    color_mode="rgb",
    batch_size=32,
    image_size=(299, 299),
    seed=123,
    validation_split=0.2,
    subset="training")
#en el fondo se podrian repetir imagenes entre conjuntos, lo mejor que se puede hacer con este codigo
#segun documentacion subset solo puede ser: One of "training" or "validation"
test_ds =tf.keras.preprocessing.image_dataset_from_directory(
    path_db,
    labels= "inferred",
    label_mode="binary",
    class_names=class_names,
    color_mode="rgb",
    batch_size=32,
    image_size=(299, 299),
    seed=123,
    validation_split=0.2,
    subset="validation")


#cada elemento de este objeto es un batch
#data preprocessing
    #agregar lo del escalamiento entre [-1,1]
    #x=keras.layers.Rescaling(scale=1.0/255)(inputs)
    #--------------------------------------------------------------
    # Pre-trained Xception weights requires that input be scaled
    # from (0, 255) to a range of (-1., +1.), the rescaling layer
    # outputs: `(inputs * scale) + offset`
    #scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
    #x = scale_layer(x)
    #-------------------------------------------------------------------

######################################################################################
#model creation
# create the base pre-trained model
base_model = keras.applications.InceptionV3(weights= None, include_top=False, input_shape=(299,299,3))
# add a global spatial average pooling layer
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = keras.layers.Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes

predictions = keras.layers.Dense(1, activation='sigmoid')(x)

#instanciando el modelo
clf_tejTumoral = keras.Model(inputs=base_model.input, outputs=predictions)
######################################################################################
#ver resumen del modelo
#clf_tejTumoral.summary()
#Model training 
    #model compilation
clf_tejTumoral.compile(optimizer=keras.optimizers.SGD(learning_rate=1e-4,momentum=0.9),
                       loss=keras.losses.BinaryCrossentropy(),
                       metrics=['accuracy'])
#entrenamiento
epochs=60
history = clf_tejTumoral.fit(train_ds, epochs=epochs,validation_data=test_ds)

#prediction

#y_pred=clf_tejTumoral(x_nuevo)
print("--- tiempo de entrenamiento: %s minutos ---" % int((time.time() - start_time)/60) )


Found 3985 files belonging to 2 classes.
Using 3188 files for training.
Found 3985 files belonging to 2 classes.
Using 797 files for validation.
Epoch 1/30
  2/100 [..............................] - ETA: 1:06:40 - loss: 0.7652 - accuracy: 0.5156

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Test Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Test Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Test Loss')
plt.legend(loc='upper right')
plt.title('Training and Test Loss')
plt.show()
plt.savefig('grafico_evaluacion_biclase.png')

In [ ]:
import numpy as np
import tensorflow as tf 
from tensorflow import keras

#data loading
path_db='/content/drive/MyDrive/memoria/db_biclase_tejtumoral'
#class_names=['tejidotumoral','tejidonotumoral']
dataset_tumoral_noTumoral = keras.preprocessing.image_dataset_from_directory(path_db,image_size=(299,299),batch_size=32, class_names=['tejidotumoral','tejidonotumoral'])
#cada elemento de este objeto es un batch

Found 1890 files belonging to 2 classes.


In [ ]:
len(dataset_biclase)#(1890/32)=59.009389183 se aproxima a 60 grupos de imagenes o batches

60

In [ ]:
for data,label in dataset_biclase:
    print(data.shape)  # (64, 200, 200, 3)
    print(data.dtype)  # float32
    print('---------------------------------------------')
    print(label)
    print(label.shape)
    print(label.dtype) #sale una lista de 32 ceros y unos porque es la clase de casa elemento dentro de un batch
    print('---------------------------------------------')
    break

(32, 299, 299, 3)
<dtype: 'float32'>
---------------------------------------------
tf.Tensor([0 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 1 0 1], shape=(32,), dtype=int32)
(32,)
<dtype: 'int32'>
---------------------------------------------
